In [1]:
from multitask import *
import botorch
from botorch.models import MultiTaskGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.random import default_rng
from sklearn.preprocessing import OneHotEncoder
import random
from pathlib import Path

In [2]:
n_t1 = 10
n_t2 = 20
use_fixed_noise=False

In [3]:
rng = default_rng(0)
rng.choice(["a", "b", "c", "d", "e", "f", "g", "h"], n_t1)

array(['g', 'f', 'e', 'c', 'c', 'a', 'a', 'a', 'b', 'g'], dtype='<U1')

## Sample Function

In [4]:
f1 = lambda X: torch.cos(5 * X[:, 0]) ** 2
f2 = lambda X: 1.5 * torch.cos(5 * X[:, 0]) ** 2
noise = 0.0
gen_inputs = lambda n: torch.rand(n, 1)
gen_obs = lambda X, f, noise: f(X) + noise * torch.rand(X.shape[0])

In [5]:
## Data Generation
X1, X2 = gen_inputs(n_t1), gen_inputs(n_t2)
i1, i2 = torch.zeros(n_t1, 1), torch.ones(n_t2, 1)

train_X = torch.cat([torch.cat([X1, i1], -1), torch.cat([X2, i2], -1)])

train_Y_f1 = gen_obs(X1, f1, noise)
train_Y_f2 = gen_obs(X2, f2, noise)
train_Y = torch.cat([train_Y_f1, train_Y_f2]).unsqueeze(-1)
train_Y_mean = train_Y.mean()
train_Y_std = train_Y.std()
train_Y_norm = (train_Y - train_Y_mean) / train_Y_std
train_Yvar = noise * torch.rand(train_X.shape[0], 1)

In [6]:
### Model Training
if not use_fixed_noise:
    model = botorch.models.MultiTaskGP(train_X, train_Y_norm, task_feature=-1)
else:
    model = botorch.models.FixedNoiseMultiTaskGP(
        train_X, train_Y_norm, train_Yvar, task_feature=-1
    )
mll = ExactMarginalLogLikelihood(model.likelihood, model)
botorch.fit.fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): MultiTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
    (task_covar_module): IndexKernel(
      (raw_var_constraint): Positive()
    )
  )
)

/Users/Kobi/Library/Caches/pypoetry/virtualenvs/multitask-k0YbXS9f-py3.9/lib/python3.9/site-packages/gpytorch/lazy/lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution
/Users/Kobi/Library/Caches/pypoetry/virtualenvs/multitask-k0YbXS9f-py3.9/lib/python3.9/site-packages/gpytorch/utils/cholesky.py:38: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/Kobi/Library/Caches/pypoetry/virtualenvs/multitask-k0YbXS9f-py3.9/lib/python3.9/site-packages/gpytorch/utils/chol

In [7]:
### Plotting
fig, ax = plt.subplots(1)

# Ground truth
X_plot = np.atleast_2d(np.linspace(0, 1, 100)).T
X_plot = torch.tensor(X_plot).float()
ax.plot(X_plot, f1(X_plot), "--", label="Task 1", alpha=1, c="r")
ax.plot(X_plot, f2(X_plot), "--", label="Task 2", alpha=1, c="b")

# Observations
ax.scatter(X1, train_Y_f1, c="r")
ax.scatter(X2, train_Y_f2, c="b")

# Posterior of model
with torch.no_grad():
    posterior = model.posterior(X_plot)
    for i in range(100):
        y = posterior.sample()[0, :, 0] * train_Y_std + train_Y_mean
        ax.plot(X_plot, y, alpha=0.01, color="r")
    for i in range(100):
        y = posterior.sample()[0, :, 1] * train_Y_std + train_Y_mean
        ax.plot(X_plot, y, alpha=0.01, color="b")

# Plot formatting
ax.legend(loc="best")

In [12]:
torch.save(model.state_dict(), "test.pth")

## Mixed Multitask Model

In [24]:
f1 = lambda X: X[:,1]*torch.cos(5 * X[:, 0]) ** 2
f2 = lambda X: X[:,1]*1.5 * torch.cos(5 * X[:, 0]) ** 2
gen_inputs_cat = lambda n, c: torch.cat([torch.rand(n, 1), torch.randint(high=c,size=(n,1))],dim=1) 
gen_inputs = lambda n: torch.rand(n, 1)
gen_obs = lambda X, f, noise: f(X) + noise * torch.rand(X.shape[0])

In [25]:
## Data Generation

X1, X2 = gen_inputs_cat(n_t1, 5), gen_inputs_cat(n_t2,5)
i1, i2 = torch.zeros(n_t1, 1), torch.ones(n_t2, 1)

train_X = torch.cat([torch.cat([X1, i1], -1), torch.cat([X2, i2], -1)])

train_Y_f1 = gen_obs(X1, f1, noise)
train_Y_f2 = gen_obs(X2, f2, noise)
train_Y = torch.cat([train_Y_f1, train_Y_f2]).unsqueeze(-1)
train_Y_mean = train_Y.mean()
train_Y_std = train_Y.std()
train_Y_norm = (train_Y - train_Y_mean) / train_Y_std
train_Yvar = noise * torch.rand(train_X.shape[0], 1)

In [27]:
### Model Training
model = MixedMultiTaskGP(train_X, train_Y_norm, task_feature=-1, cat_dims=[1])
mll = ExactMarginalLogLikelihood(model.likelihood, model)
botorch.fit.fit_gpytorch_model(mll)

/Users/Kobi/Library/Caches/pypoetry/virtualenvs/multitask-k0YbXS9f-py3.9/lib/python3.9/site-packages/botorch/fit.py:128: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(w.message, w.category)
/Users/Kobi/Library/Caches/pypoetry/virtualenvs/multitask-k0YbXS9f-py3.9/lib/python3.9/site-packages/botorch/fit.py:128: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(w.message, w.category)


ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-05)
    )
  )
  (model): MixedMultiTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-05)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): AdditiveKernel(
      (kernels): ModuleList(
        (0): ScaleKernel(
          (base_kernel): AdditiveKernel(
            (kernels): ModuleList(
              (0): MaternKernel(
                (raw_lengthscale_constraint): Positive()
                (distance_module): Distance()
              )
              (1): ScaleKernel(
                (base_kernel): CategoricalKernel(
                  (raw_lengthscale_constraint): Positive()
                )
                (raw_outputscale_constraint): Positive()
              )
  

In [29]:
torch.save(model.state_dict(), "test.pth")

## Check Suzuki

In [29]:
exp = SuzukiEmulator.load(model_name="baumgartner_suzuki", save_dir="../data/baumgartner_suzuki/emulator")
ct_data_paths = [Path("../data/reizman_suzuki/ord/reizman_suzuki_case_1.pb")]
ds_list = [
    get_suzuki_dataset(
        ct_data_path,
        split_catalyst=exp.split_catalyst,
        print_warnings=False,
    )
    for ct_data_path in ct_data_paths
]
for i, ds in enumerate(ds_list):
    ds["task", "METADATA"] = i
big_ds = pd.concat(ds_list)

In [36]:
big_ds["reagent_concentration"].max()

0.2664000138745991

In [37]:
big_ds["reagent_concentration"].min()

0.13916418346601325